In [1]:
!pip install pypd
!pip install sentence_transformers
!pip install faiss-gpu
!pip install langchain-huggingface
!pip install rag-evaluator
!pip install langchain-community langchain-core
!pip install langchain-community
!pip install transformers
!pip install transformers_stream_generator
!pip install tiktoken

  Preparing metadata (setup.py) ... done
  Created wheel for pypd: filename=pypd-1.1.0-py3-none-any.whl size=34261 sha256=32b654040ca090ae737d2d25a22b5884c09539927b626ebebc797347e52fea0c
  Stored in directory: /root/.cache/pip/wheels/55/8c/f9/9c477b62ba9c4d287fb1a841634f3c7c245c175b1ee4cca48c
Successfully built pypd
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 5.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requ

In [2]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline


In [3]:
import os
import shutil


output_dir = "migration"
os.makedirs(output_dir, exist_ok=True)


# Paths to the uploaded PDF files in the Kaggle dataset
# Update the dataset name below with the actual dataset folder name on Kaggle
dataset_dir = "/kaggle/input/migration"
pdf_files = ["FTU_4.2.3.pdf", "iml_16.pdf"]

# Copy files from the Kaggle dataset directory to the local directory
for pdf_file in pdf_files:
    source_path = os.path.join(dataset_dir, pdf_file)
    destination_path = os.path.join(output_dir, pdf_file)
    shutil.copy(source_path, destination_path)

print(f"PDF files have been copied to {output_dir}")


PDF files have been copied to migration


In [4]:
# Load pdf files in the local directory
loader = PyPDFDirectoryLoader("./migration/")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(metadata={'source': 'migration/FTU_4.2.3.pdf', 'page': 0}, page_content='Fact Sheets on the European Union - 2024 1\nwww.europarl.europa.eu/factsheets/en\nIMMIGRATION POLICY\nA forward-looking and comprehensive European immigration policy, based on\nsolidarity, is a key objective for the European Union. Immigration policy is intended to\nestablish a balanced approach to dealing with both regular and irregular immigration.\nLEGAL BASIS\nArticles 79 and 80 of the Treaty on the Functioning of the European Union (TFEU).\nCOMPETENCES\nRegular immigration : The EU has the authority to establish the conditions for entry\nand legal residence in a Member State, including for family-reunification purposes,')

In [5]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs]) // len(docs)

# Calculate the average document length before and after the split
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

# Print the results in the desired format
print(f"Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.")
print(f"After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split}.")


Before split, there were 648 documents loaded, with average characters equal to 2622.
After split, there were 3363 documents (chunks), with average characters equal to 510.


In [7]:
huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(docs_after_split, huggingface_embeddings)

'''
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(docs, embedding_model) 
 '''


'\nembedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")\ndb = FAISS.from_documents(docs, embedding_model) \n '

In [8]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-3.31904413e-03 -9.44509695e-04  1.08670210e-02 -2.16592196e-02
 -7.60440007e-02 -1.22209359e-02  8.43056738e-02 -1.25443731e-02
  6.66513219e-02  4.45139920e-03  1.16965123e-01 -4.58721444e-03
 -6.59863651e-02  7.35700503e-03  1.25031322e-02  7.47561455e-03
  3.57134119e-02  1.14197633e-03 -6.93238378e-02 -2.19857246e-02
 -2.37161126e-02  2.18495224e-02  4.15965915e-03 -9.19143669e-03
  1.23697612e-02  7.87825957e-02  1.21582439e-02 -1.25874467e-02
 -1.89647842e-02  1.74058601e-02  3.05591412e-02 -2.33882759e-03
  6.45622751e-03 -2.47534681e-02  1.98177077e-06 -3.91909853e-02
 -4.51903651e-03  2.57006977e-02  1.72387008e-02 -5.31584546e-02
 -3.09361350e-02 -6.36262596e-02 -2.91672163e-02  9.35038179e-03
  2.31869947e-02  1.82543881e-02 -2.55351663e-02 -1.44439228e-02
  4.64899391e-02  3.03460229e-02  2.74518831e-03 -5.37015721e-02
 -5.23337796e-02 -2.28478815e-02 -6.52508885e-02  3.74247730e-02
  1.00855920e-02 -8.10862482e-02 -3.23853455e-02 -4

In [9]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)


In [10]:
query = """what are the immigration requirments for germany"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

of the issues connected to the migration policy; cooperating on creating and realizing integration policy towards immigrants; and initiating, analyzing and preparing opinions on draft laws and other documents relating to the migration policy.Poland


In [9]:
# Step 3: Load the question-answering model for answer generation
# Using a smaller variant of T5 (flan-t5-large) to optimize for faster response generation
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map="auto")
question_answerer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [28]:
# Step 4: Define the retrieval and answer generation function
def retrieve_and_answer(query):
    # Retrieve the top 3 most relevant chunks based on similarity to the query
    contexts = db.similarity_search(query, k=3)
    
    # Check if any relevant chunks are found
    if not contexts:
        print(f"Question: {query}\nAnswer: Sorry, your question could not be answered based on the documents stored in my database.\n")
        return

    # Combine the retrieved chunks into a single context
    combined_context = " ".join([context.page_content for context in contexts])
    
    # If combined context is empty, return a default response
    if not combined_context.strip():
        print(f"Question: {query}\nAnswer: Sorry, your question could not be answered based on the documents stored in my database.\n")
        return

    # Generate a concise answer based on the retrieved context
    prompt = f"Answer concisely: {combined_context}\nQuestion: {query}\nAnswer:"
    
    # Generate answer with a controlled response length for efficiency
    response = question_answerer(prompt, max_length=200, early_stopping=True)
    answer = response[0]['generated_text'].strip()
    print(f"Question: {query}\nAnswer: {answer}\n")


In [24]:
questions = [
    "What are the general immigration requirements for newcomers in Germany?",
    "How long does the visa application process take in Germany?",
    "What are the family reunification policies in Germany?",
    "Are there any specific language requirements to migrate to Germany?",
    "What types of work visas are available in Germany?",
    "Can I apply for permanent residency in Germany after a certain period?",
    "what is capital of pakistan"
]


In [29]:
# Run the pipeline for each question to retrieve and answer
for question in questions:
    retrieve_and_answer(question)

Question: What are the general immigration requirements for newcomers in Germany?
Answer: • Proof of sufficient financial means • Health insurance • Basic knowledge of the German language • Third-country national must sign a declaration that he is aware that the permit can be refused and expulsion can be ordered if false information is provided • Other documents necessary to prove the purpose of immigration

Question: How long does the visa application process take in Germany?
Answer: three months

Question: What are the family reunification policies in Germany?
Answer: Spouses and minor children of German citizens may be issued a residence permit for the purpose of family reunification, as may the parents of minor unmarried Germans for the purpose of care

Question: Are there any specific language requirements to migrate to Germany?
Answer: Knowledge of the German language is not required prior to entry

Question: What types of work visas are available in Germany?
Answer: seasonal wor

## Answers which are provided in document are good but irrelevant questions are also being answered but that's just totally giberish